<a href="https://colab.research.google.com/github/swastipriya/Deep-Learning/blob/master/OpenVino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import glob
import torch.nn as nn
import os.path as osp
import torchvision
import matplotlib.pyplot as plt
import torch
#import rotateImage
import cv2
import numpy as np
#import split_folders
import random
import torch.backends.cudnn as cudnn
import torch.optim as optim
from random import choice
from collections import OrderedDict
from torchsummary import summary

In [3]:
the_model = torch.load('/content/drive/My Drive/00_openvino/resnet50_places365.pth.tar')

In [4]:
type(the_model)

dict

In [12]:
print(the_model.keys())

dict_keys(['state_dict', 'epoch', 'arch', 'best_prec1'])


In [15]:
the_model['epoch']

66

In [16]:
the_model['arch']

'resnet50'

In [17]:
the_model['best_prec1']

55.57260273972603

In [9]:
def load_model(checkpoint_path):
  chpt = torch.load(checkpoint_path)
  net = torchvision.models.resnet50(pretrained=True)
  # fc = nn.Sequential(OrderedDict([
  #   ("dropout", nn.Dropout(0.4)),
  #   ("out", nn.Linear(2048, 11))]))
  # net.fc = fc

  for param, x in net.named_parameters():
    if param.startswith("fc.out"):
        x.requires_grad = True
    else:
        x.requires_grad = False

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  #net.class_to_idx = classes
  net = net.to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
    
  # net.class_to_idx = chpt['class_to_idx']
    
     
    
  net.load_state_dict(chpt['state_dict'], strict=False)
    
  return net

In [10]:
the_model = load_model('/content/drive/My Drive/00_openvino/resnet50_places365.pth.tar')

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [11]:
type(the_model)

torchvision.models.resnet.ResNet

In [15]:
print(the_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [26]:
for name, param in the_model.named_parameters():
    print(name, ':', param.requires_grad)

conv1.weight : False
bn1.weight : False
bn1.bias : False
layer1.0.conv1.weight : False
layer1.0.bn1.weight : False
layer1.0.bn1.bias : False
layer1.0.conv2.weight : False
layer1.0.bn2.weight : False
layer1.0.bn2.bias : False
layer1.0.conv3.weight : False
layer1.0.bn3.weight : False
layer1.0.bn3.bias : False
layer1.0.downsample.0.weight : False
layer1.0.downsample.1.weight : False
layer1.0.downsample.1.bias : False
layer1.1.conv1.weight : False
layer1.1.bn1.weight : False
layer1.1.bn1.bias : False
layer1.1.conv2.weight : False
layer1.1.bn2.weight : False
layer1.1.bn2.bias : False
layer1.1.conv3.weight : False
layer1.1.bn3.weight : False
layer1.1.bn3.bias : False
layer1.2.conv1.weight : False
layer1.2.bn1.weight : False
layer1.2.bn1.bias : False
layer1.2.conv2.weight : False
layer1.2.bn2.weight : False
layer1.2.bn2.bias : False
layer1.2.conv3.weight : False
layer1.2.bn3.weight : False
layer1.2.bn3.bias : False
layer2.0.conv1.weight : False
layer2.0.bn1.weight : False
layer2.0.bn1.bias : 

In [27]:
dummy_input = torch.randn(1, 3, 224, 224, device='cuda')
onnx_path =  "/content/drive/My Drive/00_openvino/pytorch_resnet50_places365.onnx"

torch.onnx.export(the_model, dummy_input, onnx_path, verbose=False)
print('Model exported to ' + onnx_path)

Model exported to /content/drive/My Drive/00_openvino/pytorch_resnet50_places365.onnx
